In [16]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [17]:
#############################
### Essentials functions ###
#############################
import FunctionMain as fm

In [18]:
# ----------------- Parameters

# The file to consider
file = './file_with_optimal_val/la04.txt'

# The number of solutions that we will have in the solver during the first iteration
k = 10

# The number of layer which is fixed
nb_layers = 5

# Variable which display the solution
display_sol = True

# Variable which display the start (in a vector)
display_start = False

# Variable which display the start (in a matrix)
display_matrix = False

# Time stop criterion 
tps_max = 1000

# Number of iteration stop criterion
it_max = 10

# Initialization of the number of iteration
it = 1

# The number of solutions that we will have in the solver after the first iteration
k_k = 10

list_search_type = ["DepthFirst", "Restart", "MultiPoint", "IterativeDiving"]

In [19]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# --------- Interaction with the solver
data = []
n, m, data, T_machine, T_duration, duration, optimalval = fm.get_data_from_file(file)

# --------- Call Solver constructor in Solver.py and create the variables of the model
model, solver = fm.initialize_solver(data, n, m, duration)

# ------------ Solve the model
print("\nSolving the model...")

list_variables = solver.get_variables()
variables = list_variables[0]

# ------------ Add constraints to the solver

print("\nAdding precedence constraints to the solver...")
# Add precedence constraints
for i in range(n):
    for j in range(1,m):
        solver.add_constraint(model, end_before_start(variables[i][T_machine[i*m + j-1]], variables[i][T_machine[i*m + j]]))
print("Precedence constraints added !")

print("\nAdding disjunctive constraints to the solver...")
# Add disjunctive constraints 
for machine in range(m):
    solver.add_constraint(model, no_overlap([variables[i][machine] for i in range(n)]))
print("Disjunctive constraints added !")

# Add objective function
waiting_time = [[] for i in range(m)]
machines = [[] for i in range(m)]
list_obj = []

for j in range(m):
    for i in range(n):
        print(variables[i][j])
        machines[j].append(variables[i][j])
    # print(machines[j])
    machines[j] = sorted(machines[j], key=lambda k: model.start_of(k))

for j in range(m):
    for i in range(len(machines[j])-1):
        waiting_time[j].append(model.start_of(machines[j][i+1]) - model.end_of(machines[j][i]))



for machine in range(m):
    sum = 0
    for i in range(len(waiting_time)-1):
        for j in range(i+1, len(waiting_time[i])):
            sum += abs(waiting_time[machine][j] - waiting_time[machine][i])
    list_obj.append(sum)

sum = 0
for i in range(len(list_obj)):
    sum += list_obj[i]

solver.add_constraint(model, min(sum))


# Solve the model.
msol = model.solve()

# ------------ Display the result
fm.display_solution(msol, display_sol)
print("Model solved !")

# # ---------------- Interaction with the user
# print("\n--------Interaction with the user...---------")
# print("\nCreating the user...")
# user = User()
# print("User created !")

# #Get the variables of the model
# variables = solver.get_variables()

# list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)

# # Vector of the start time of each task of each preference
# starts = user.start_pref(n, m, variables, display_start)

# # Matrix of the start time of each task of each preference
# matrix = user.matrix_pref(n, m, display_matrix)

# # Testing the order of preferences and the differences between solutions
# fm.test(n, m, user)

# print("list layers : ",list_layers)



--------Main program is loading...---------

Creating the model variables...
Model variables created !

Solver initialized !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !
"T0-0" = intervalVar(size=12)
"T1-0" = intervalVar(size=87)
"T2-0" = intervalVar(size=75)
"T3-0" = intervalVar(size=7)
"T4-0" = intervalVar(size=15)
"T5-0" = intervalVar(size=76)
"T6-0" = intervalVar(size=52)
"T7-0" = intervalVar(size=62)
"T8-0" = intervalVar(size=62)
"T9-0" = intervalVar(size=88)


CpoException: CPO expression can not be used as boolean.